
... ***CURRENTLY UNDER DEVELOPMENT*** ...


## RBFs reconstruction of synthetic data

inputs required: 
  * Synthetic offshore waves - emulator output
  * Sea and swell **SWAN simulated cases**

in this notebook:
  * RBF reconstruction simulated storms
  * Generation of hourly nearshore waves with Intradaily Hydrographs



### Workflow:

<div>
<img src="../resources/nb02_03.png" width="400px">
</div>



In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import pandas as pd
import xarray as xr

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..', '..'))

# teslakit
from teslakit.database import Database, hyswan_db
from teslakit.rbf import RBF_Reconstruction, RBF_Validation
from teslakit.waves import Aggregate_WavesFamilies, AWL, Intradaily_Hydrograph, Snell_Propagation
from teslakit.climate_emulator import Climate_Emulator
from teslakit.util.time_operations import repair_times_hourly, add_max_storms_mask



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('ROI')

# hyswan simulations database
db_sim = hyswan_db(db.paths.site.HYSWAN.sim)


In [3]:
# --------------------------------------
# aux.

def fix_target_nans(subset, target):
    'remove NaN data indexes from subset and target. RBF does not handle NaNs'
    
    r_nan = target.isnull().any(axis=1)  # find any row with nans

    if r_nan.any():
        # log
        print('remove nan data found at target:')
        print(target[r_nan])
        
        # clean data
        target = target[~r_nan]
        subset = subset[~r_nan]
        
    return subset, target


# Snell propagation 400m -> 21m
def snell_prop_data(waves_dset, d1, d2, ob):
    'teslakit/waves.py Snell_Propagation func. parser'
    
    # get variables
    Tp = waves_dset['TPsmoo'].values[:]
    Hs = waves_dset['Hsig'].values[:]
    Dir = waves_dset['Dir'].values[:]
  
    # TODO: check dir modifications and refactor
    dir_I = Dir.copy()

    # El angulo definido entre -90 y 90
    dir_I[np.where(dir_I > 180)[0]] = dir_I[np.where(dir_I > 180)[0]] - 360

    # Obligamos que el angulo este en este sector
    dir_I[np.where(dir_I > 90)[0]] = 85
    dir_I[np.where(dir_I <= -90)[0]] = -85

    Hs_p, Dir_p, ks, kp = Snell_Propagation(Tp, Hs, dir_I, d1, d2, ob)
    
    # return copy of input
    out = waves_dset.copy()
    out['Hsig'] = Hs_p
    out['Dir'] = Dir_p
    
    return out



## SIMULATION Waves Reconstruction

In [4]:
# --------------------------------------
# RBF Reconstruction data

# sea
wvs_sea_dataset = db_sim.Load('sea_dataset') 
wvs_sea_subset = db_sim.Load('sea_subset') 
wvs_sea_target = db_sim.Load('sea_target') 

# swells
wvs_swl_dataset = db_sim.Load('swl_dataset') 
wvs_swl_subset = db_sim.Load('swl_subset') 
wvs_swl_target = db_sim.Load('swl_target') 


In [5]:
# --------------------------------------
# prepare dataset, subset and target data (propagated waves)


# solve Hs coefficient
wvs_sea_target['Hsig'] = wvs_sea_target['Hsig'] * wvs_sea_subset['hs']
wvs_swl_target['Hsig'] = wvs_swl_target['Hsig'] * wvs_swl_subset['hs']

# Snell propagation 
depth_1, depth_2, bathy_dir = 400, 21, 0
wvs_sea_target = snell_prop_data(wvs_sea_target, depth_1, depth_2, bathy_dir)
wvs_swl_target = snell_prop_data(wvs_swl_target, depth_1, depth_2, bathy_dir)


# remove nans (if any) from subset and target
wvs_sea_subset, wvs_sea_target = fix_target_nans(wvs_sea_subset, wvs_sea_target)
wvs_swl_subset, wvs_swl_target = fix_target_nans(wvs_swl_subset, wvs_swl_target)


# ensure dataset and subset have same variables and column order
vns_ds = ['hs', 'tp', 'dir']

wvs_sea_dataset = wvs_sea_dataset[vns_ds].values
wvs_sea_subset = wvs_sea_subset[vns_ds].values

wvs_swl_dataset = wvs_swl_dataset[vns_ds].values
wvs_swl_subset = wvs_swl_subset[vns_ds].values


# select target variables
vns_tgt = ['Hsig', 'TPsmoo', 'Dir']

wvs_sea_target = wvs_sea_target[vns_tgt].values
wvs_swl_target = wvs_swl_target[vns_tgt].values


remove nan data found at target:
         lat       lon  Hsig  Tm02  Dir  Dspr  TPsmoo
case                                                 
103   9.4165  167.4679   NaN   NaN  NaN   NaN     NaN


In [6]:
# --------------------------------------
# RBF Reconstruction library

# subset - scalar / directional indexes
ix_scalar_subset = [0,1]      # scalar (hs, tp)
ix_directional_subset = [2]   # directional (dir)

# target - scalar / directional indexes
ix_scalar_target = [0,1]      # scalar (Hsig, Tpsmoo, Dir)
ix_directional_target = [2]   # directional (Dir)


# RBF wrappers 
def RBF_Reconstruction_sea(dataset_input):
    
    return RBF_Reconstruction(
    wvs_sea_subset, ix_scalar_subset, ix_directional_subset,
    wvs_sea_target, ix_scalar_target, ix_directional_target,
    dataset_input)

def RBF_Reconstruction_swell(dataset_input):
    
    return RBF_Reconstruction(
    wvs_swl_subset, ix_scalar_subset, ix_directional_subset,
    wvs_swl_target, ix_scalar_target, ix_directional_target,
    dataset_input)

def RBF_Reconstruction_families(data):
   
    # sea
    vs = ['sea_Hs', 'sea_Tp', 'sea_Dir']
    data_sea = data[vs].dropna()
    data_sea.drop(data_sea[(data_sea['sea_Dir'] >= 360)].index, inplace=True)  # fix sea_Dir >> 360 bug    
    rec_sea = RBF_Reconstruction_sea(data_sea.values)
    rec_sea = pd.DataFrame(data=rec_sea, columns=vs, index=data_sea.index)

    # swell 1
    vs = ['swell_1_Hs', 'swell_1_Tp', 'swell_1_Dir']
    data_swl_1 = data[vs].dropna()
    rec_swl_1 = RBF_Reconstruction_swell(data_swl_1.values)
    rec_swl_1 = pd.DataFrame(data=rec_swl_1, columns=vs, index=data_swl_1.index)

    # swell 2
    vs = ['swell_2_Hs', 'swell_2_Tp', 'swell_2_Dir']
    data_swl_2 = data[vs].dropna()
    rec_swl_2 = RBF_Reconstruction_swell(data_swl_2.values)
    rec_swl_2 = pd.DataFrame(data=rec_swl_2, columns=vs, index=data_swl_2.index)
    
    # join nearshore reconstructed data and parse to xarray.Dataset
    rec_waves = pd.concat([rec_sea, rec_swl_1, rec_swl_2], axis=1)

    # return pandas.dataframe
    return rec_waves


In [7]:
# --------------------------------------
# Load simulation waves families

n_sims_DWTs = 10

CE = Climate_Emulator(db.paths.site.EXTREMES.climate_emulator)
CE.Load()

WVS = CE.LoadSim_All()

# Reconstruct waves by families
WVS_rec = RBF_Reconstruction_families(WVS)


# set indexes values
WVS_rec['n_sim'] = WVS.n_sim
WVS_rec['time'] = WVS.time

# store reconstruction
db_sim.Save('reconstruction', WVS_rec)


ix_scalar: 0,  optimization: 2.47 | interpolation: 98.80
ix_scalar: 1,  optimization: 5.44 | interpolation: 101.32
ix_directional: 2,  optimization: 5.08 | interpolation: 198.86
ix_scalar: 0,  optimization: 1.96 | interpolation: 94.56
ix_scalar: 1,  optimization: 4.95 | interpolation: 96.78
ix_directional: 2,  optimization: 4.74 | interpolation: 190.89
ix_scalar: 0,  optimization: 1.90 | interpolation: 65.22
ix_scalar: 1,  optimization: 6.73 | interpolation: 64.40
ix_directional: 2,  optimization: 5.29 | interpolation: 114.85


In [8]:
def waves_hourly(CE, WVS_SIM_rec, n_sim_ce=0):
    '''
    Generate nearshore hourly simulation output. 
    
    - aggregate nearshore waves storms
    - generate hourly hydrographs
    - calculates AWL
    
    Stores simulation nearshore waves at teslakit site database
    '''
    
    # iterate simulations
    for n in range(n_sims_DWTs):
        
        # select simulation
        WVS_SIM_rec_n = WVS_SIM_rec.loc[WVS_SIM_rec['n_sim'] == n]

        # aggregate nearshore variables (Tp from max. energy waves family)
        wvs_agr = Aggregate_WavesFamilies(WVS_SIM_rec_n, a_tp='max_energy')                
        
        # Load TCs storm data from climate emulator
        _, TCS_sim, _ = CE.LoadSim(n_sim = n)    
        TCS_sim = TCS_sim.isel(n_sim = n_sim_ce).copy() 
        TCS_sim = TCS_sim.sel(time=wvs_agr.time)
        
        # calculate intradaily hourly hydrographs for simulated storms
        wvs_h = Intradaily_Hydrograph(wvs_agr, TCS_sim)
        
        # calculate AWL
        wvs_h['AWL'] = AWL(wvs_h.Hs, wvs_h.Tp)
        
        # store hourly simulation offshore data
        db.Save_SIM_NEARSHORE(wvs_h, n)
                
        print('simulation {0} processed.'.format(n))

# generate intradaily hydrographs
waves_hourly(CE, WVS_rec)  


simulation 0 processed.
simulation 1 processed.
simulation 2 processed.
simulation 3 processed.
simulation 4 processed.
simulation 5 processed.
simulation 6 processed.
simulation 7 processed.
simulation 8 processed.
simulation 9 processed.
